In [1]:
import re
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [2]:
Career_stats = pd.read_html("https://www.nhl.com/player/auston-matthews-8479318")
Career_stats = Career_stats[0]
Career_stats

,GP,G,A,P,+/-
0,71,38,42,80,29


In [58]:
webpage = requests.get("https://www.nhl.com/player/8479318")
webpage = bs(webpage.content, 'html.parser')


x = webpage.find("span", text="Draft:")
y = webpage.find(class_ = "player-jumbotron-vitals--attr")
y = str(y)
y = y[-8:-7]
print(y)

C


In [22]:
webpage = requests.get("https://www.nhl.com/player/sebastian-aho-8478427")
webpage = bs(webpage.content, 'html.parser')

Birthdata = webpage.find("span", text="Born:").next_sibling
Birthyear = Birthdata.split(',')[1]
Birthyear = ''.join(c for c in Birthyear if c.isdigit())
BirthMonth = Birthdata.split(',')[0]
BirthMonth = ''.join(c for c in BirthMonth if c.isalpha())
BirthMonth

'July'

In [42]:
Draft_info = webpage.find("span", text="Draft:").next_sibling
Draft_Position = Draft_info.split('(')[1]
Draft_Position = ''.join(c for c in Draft_Position if c.isdigit())
Draft_Year = Draft_info.split(',')[0]
Draft_Team = ''.join(c for c in Draft_Year if c.isalpha())
Draft_Year = ''.join(c for c in Draft_Year if c.isdigit())

Draft_Team =  webpage.find("span", text="Draft:").next_sibling

'CAR'

In [59]:
Position = webpage.find(class_ = "player-jumbotron-vitals--attr")
Position = str(Position)
Position = Position[-8:-7]
Position

'C'

In [ ]:
Handedness

In [5]:
webpage = requests.get("https://www.nhl.com/player/sebastian-aho-8478427")
webpage = bs(webpage.content, 'html.parser')

Birthyear = webpage.find("span", text="Born:").next_sibling
Birthyear = Birthyear.split(',')[1]
Birthyear = ''.join(c for c in Birthyear if c.isdigit())


Draft_info = webpage.find("span", text="Draft:").next_sibling
Draft_info = Draft_info.split('(')[1]
Draft_info = ''.join(c for c in Draft_info if c.isdigit())


data = {'Player_ID': [8478427],
        'Born': [Birthyear],
        'Drafted': [Draft_info]}

Bio_DF = pd.DataFrame(data)
Bio_DF

,Player_ID,Born,Drafted
0,8478427,1997,35


In [79]:
def get_player_bios(Player_ID_list):
    n=1
    m = len(Player_ID_list)
    for i in Player_ID_list:
        t1 = time.time()
        webpage = requests.get(f"https://www.nhl.com/player/{i}")
        webpage = bs(webpage.content, 'html.parser')
        ###
        Check_for_Dob = webpage.find("span", text="Born:")
        if Check_for_Dob==None:
            Birthyear='missing'
            BirthMonth='missing'
        else:
            Birthdata = webpage.find("span", text="Born:").next_sibling
            Birthyear = Birthdata.split(',')[1]
            Birthyear = ''.join(c for c in Birthyear if c.isdigit())
            BirthMonth = Birthdata.split(',')[0]
            BirthMonth = ''.join(c for c in BirthMonth if c.isalpha())
        
        ###
        Check_for_draft = webpage.find("span", text="Draft:")
        if Check_for_draft==None:
            Draft_Year='undrafted'
            Draft_Team='undrafted'
            Draft_Position='undrafted'
            
        else:
            Draft_info = webpage.find("span", text="Draft:").next_sibling
            Draft_Position = Draft_info.split('(')[1]
            Draft_Position = ''.join(c for c in Draft_Position if c.isdigit())
            Draft_Year = Draft_info.split(',')[0]
            Draft_Team = ''.join(c for c in Draft_Year if c.isalpha())
            Draft_Year = ''.join(c for c in Draft_Year if c.isdigit())
        ###
        check_for_position = webpage.find(class_ = "player-jumbotron-vitals--attr")
        if check_for_position==None:
            Position = missing
        else:
            Position = webpage.find(class_ = "player-jumbotron-vitals--attr")
            Position = str(Position)
            Position = Position[-8:-7]
        
        ###
        check_for_handedness = webpage.find("span", text="Shoots:")
        if check_for_handedness==None:
            Handedness = missing
        else:
            Handedness = webpage.find("span", text="Shoots:").next_sibling
            Handedness = ''.join(c for c in Handedness if c.isalpha())
        ###
        data = {'Player_ID': [i],
                'Position': [Position],
                'Handedness': [Handedness],
                'Birth_Year': [Birthyear],
                'Birth_Month': [BirthMonth],
                'Draft_Year': [Draft_Year],
                'Draft_Team': [Draft_Team],
                'Draft_Position': [Draft_Position]}
        Bio_DF = pd.DataFrame(data)
        
        if i == Player_ID_list[0]:
            Bio_DF_Final = Bio_DF
        else:
            Bio_DF_Final = pd.concat([Bio_DF_Final, Bio_DF])
        
        t2 = time.time()
        print(f'Took {t2-t1} seconds to retrieve data for player {i}')
        print(f'{m-n} player(s) remain')
        time.sleep(2)
        n+=1
    return Bio_DF_Final
        

In [69]:
#if not Check_for_draft:
#    Draft_info='undrafted'

# add sleep not to get noticed
# functionality to save a file every x attempts is possible and can prevent disasters.


#Alternate check for draft position.

In [80]:
Player_Bios = get_player_bios([8470151,8470966])
Player_Bios

Took 2.9315648078918457 seconds to retrieve data for player 8470151
1 player(s) remain
Took 2.2011802196502686 seconds to retrieve data for player 8470966
0 player(s) remain


,Player_ID,Position,Handedness,Birth_Year,Birth_Month,Draft_Year,Draft_Team,Draft_Position
0,8470151,D,Left,1983,September,2002,FLA,3
0,8470966,D,Left,1983,October,undrafted,undrafted,undrafted


In [81]:
player_id_list = [8470638,8478402,8477409,8479314,8480835,8480830,8475913,8477933,8478233,8475848,8477210,8483808,8476461,8482730,8475738,8479318,8476453,8475726,8471675,8480027,8479591,8476346,8467514,8479414,8477478,8481656,8478427,8480801,8477949,8478115,8478414,8476798,8477511,8478975,8478483,8478493,8476399,8471228,8480443,8479323,8478864,8475193,8476410,8473618,8479325,8478585,8477901,8475184,8477448,8477942,8479400,8475768,8482093,8478047,8471218,8477983,8480002,8479619,8477493,8479335,8476887,8478856,8478449,8478376,8471685,8477476,8478131,8481481,8475323,8471887,8476292,8469459,8477940,8476448,8475799,8468508,8468598,8482159,8476430,8478450,8477501,8478873,8481725,8475250,8476479,8478028,8483619,8480459,8478445,8478870,8475845,8482623,8478365,8477935,8481624,8473588,8478421,8480007,8479344,8470041,8474176,8476455,8475253,8477947,8481580,8480289,8478020,8474025,8476539,8471215,8476447,8475800,8468483,8475166,8474018,8481559,8471669,8448208,8469707,8481167,8476439,8470794,8476474,8477993,8476438,8475314,8473422,8476881,8478949,8475692,8481093,8481641,8474849,8479941,8478420,8476310,8478519,8473512,8475791,8475786,8473537,8476235,8479536,8478542,8477346,8480890,8479977,8479944,8469466,8482055,8475767,8477946,8470612,8480762,8478550,8470622,8478057,8469506,8480901,8480012,8481543,8473533,8468083,8478038,8476826,8474716,8470176,8477529,8470595,8477458,8471682,8477416,8483524,8477951,8471842,8470867,8478866,8479675,8471276,8475168,8477998,8466138,8479982,8468745,8481422,8477392,8482097,8476868,8482113,8479518,8476483,8476921,8479447,8479439,8469485,8473994,8480935,8476428,8481178,8475172,8476451,8481577,8476907,8477290,8481518,8479407,8480185,8480802,8482175,8474569,8478512,8478049,8477402,8474793,8480069,8478462,8474679,8477541,8475214,8478440,8480944,8476480,8476389,8477447,8475158,8478891,8475765,8470144,8476436,8478508,8477930,8473604,8475844,8474157,8479546,8477479,8481638,8474034,8474098,8475225,8476894,8482146,8475810,8480208,8477963,8477520,8481535,8466148,8476880,8479466,8479383,8476116,8477492,8476958,8477846,8478046,8470655,8478843,8475820,8475160,8474646,8475982,8482110,8481813,8476283,8482124,8476406,8480267,8478067,8475690,8479525,8476456,8475722,8475855,8476460,8468518,8470611,8480771,8470280,8479293,8480384,8478569,8478831,8474053,8474149,8478017,8470610,8475949,8478407,8477934,8474164,8480078,8473970,8477939,8482248,8469638,8480226,8480980,8482116,8469483,8476469,8476392,8474603,8477429,8478056,8478438,8476393,8482652,8476617,8480028,8477944,8470162,8479337,8475784,8480789,8481592,8479329,8477341,8477919,8479322,8475760,8477413,8458537,8477450,8475147,8476892,8467876,8476917,8478399,8474145,8476987,8468485,8477902,8474611,8478970,8481004,8477839,8479657,8474131,8479343,8480965,8478486,8479543,8474089,8476806,8477417,8478010,8473453,8470543,8473463,8479304,8460621,8475164,8476867,8480008,8475183,8476205,8471699,8476381,8477464,8479573,8479542,8473986,8474150,8481527,8480988,8478042,8478911,8478027,8471769,8482691,8479734,8470604,8473419,8471707,8480015,8476442,8481604,8467463,8479066,8470798,8477407,8476872,8480355,8477881,8476994,8478498,8480829,8478474,8470713,8477454,8476482,8476966,8469544,8475761,8477496,8474565,8481186,8482197,8476322,8479379,8480863,8480467,8474586,8482241,8480893,8479315,8475749,8481560,8478398,8474037,8477903,8481516,8482259,8476854,8467988,8478413,8474715,8470667,8475191,8478839,8471214,8478417,8479290,8476549,8477979,8474685,8474681,8475728,8476944,8469500,8479576,8479364,8481711,8476882,8478211,8475218,8476331,8481072,8481068,8479974,8477046,8478472,8476983,8470606,8479390,8477330,8481542,8478488,8479553,8468501,8475745,8475715,8475220,8477355,8478106,8481477,8481557,8478442,8479661,8470626,8480205,8475754,8481523,8476209,8474641,8481479,8475794,8476345,8479992,8480082,8471273,8480087,8478013,8476765,8471794,8481521,8478268,8474709,8479404,8478458,8480803,8477127,8475736,8482092,8471362,8479648,8480849,8481441,8474001,8474190,8477034,8471680,8474030,8471851,8475798,8477444,8470621,8474059,8478469,8475763,8474062,8475194,8468064,8471676,8477956,8479602,8481601,8477426,8478455,8482665,8477406,8473673,8476952,8481524,8478846,8478838,8475247,8481849,8475254,8476871,8466160,8481637,8479345,8471208,8478424,8477931,8473473,8480247,8479547,8471299,8477451,8475413,8480018,8471233,8480780,8476468,8471338,8478444,8478443,8481704,8482240,8473534,8480990,8477410,8471709,8465202,8476525,8474739,8474102,8482671,8477021,8477404,8481596,8471702,8478416,8475200,8469760,8475461,8480192,8478040,8478078,8477508,8482149,8483570,8476981,8473546,8479482,8476927,8482109,8480821,8474005,8475423,8476955,8471339,8473574,8481425,8478133,8471280,8477003,8471698,8477091,8477227,8475637,8482117,8474161,8473412,8473564,8480053,8471691,8475177,8480029,8477494,8476440,8471371,8476948,8478485,8474040,8478904,8474642,8478408,8481532,8480220,8483630,8478403,8469664,8476918,8478567,8470616,8480081,8473504,8475769,8480023,8479370,8476419,8473921,8473507,8482157,8474613,8481630,8471231,8480049,8481013,8471476,8480341,8468504,8477010,8470609,8475343,8475996,8481058,8481568,8475772,8476293,8470619,8460496,8476244,8473536,8477482,8480068,8474668,8482834,8474218,8481606,8476885,8480259,8473579,8479365,8470120,8469455,8474031,8480860,8471724,8477929,8470018,8478099,8470105,8481582,8470039,8467545,8475770,8481581,8478466,8479381,8474727,8471996,8475278,8475782,8480448,8482111,8475176,8471260,8475869,8470366,8467967,8477244,8482101,8481554,8477213,8470852,8470268,8476462,8478463,8474038,8477591,8480820,8475328,8479351,8480084,8473449,8477810,8477960,8481703,8476878,8477845,8477969,8477987,8471678,8478507,8473544,8474564,8480143,8479516,8475204,8481617,8479462,8475739,8475098,8475209,8477500,8480870,8477038,8471311,8481533,8480846,8471217,8477041,8468639,8469798,8468208,8477240,8470596,8477503,8476891,8477941,8470123,8476463,8467407,8481056,8480073,8474600,8474162,8470642,8470358,8471450,8468486,8471976,8479419,8479393,8479976,8469465,8480242,8474578,8480113,8470834,8476166,8471346,8474884,8481546,8467890,8474013,8481618,8476889,8474189,8481537,8468575,8476432,8476415,8477955,8479968,8474024,8476766,8474576,8477015,8475175,8477214,8471266,8475907,8475902,8476975,8476772,8478502,8475733,8477566,8470230,8475309,8480434,8475151,8464975,8476429,8479483,8477937,8477948,8477365,8477957,8470966,8476368,8476505,8479321,8470750,8475752,8475185,8474141,8479355,8475163,8475169,8460626,8478584,8479410,8481655,8471262,8475170,8464989,8480800,8482105,8470189,8477466,8482150,8476458,8476819,8475796,8479328,8469752,8468493,8477945,8475268,8480043,8475210,8478063,8481553,8480054,8482245,8475793,8474056,8471283,8474589,8469492,8471474,8476344,8470232,8470778,8477936,8480839,8471703,8477887,8480036,8481442,8477326,8470257,8476906,8477959,8475287,8471498,8477220,8477631,8480074,8474590,8480807,8478833,8480468,8473991,8477832,8476288,8471873,8476467,8471716,8477680,8480727,8477498,8482077,8476374,8471681,8468498,8477836,8478431,8477070,8474236,8479465,8476427,8476214,8475157,8480853,8476062,8477996,8479705,8474609,8467875,8480880,8470201,8476459,8470886,8475968,8474019,8476624,8476195,8475598,8474554,8477923,8473927,8477435,8480032,8479718,8482078,8475462,8475296,8477126,8476923,8467365,8482667,8478036,8476873,8483565,8476979,8477353,8481077,8467332,8469457,8478888,8468484,8476425,8480196,8478409,8462038,8467423,8472410,8476858,8474772,8478563,8468101,8481550,8473933,8477337,8471303,8475748,8474588,8478075,8479671,8466139,8476372,8478447,8474192,8475208,8471700,8469622,8477816,8479587,8477491,8476913,8477201,8479523,8480844,8476472,8469470,8480186,8475714,8479933,8478434,8473562,8478074,8471254,8478882,8479250,8476769,8475826,8477033,8474096,8475597,8470169,8470047,8467096,8475291,8479382,8480833,8477461,8478366,8478137,8477467,8476915,8476911,8475602,8471804,8476302,8473942,8477505,8479366,8475263,8475292,8480330,8474077,8477446,8478906,8474535,8469473,8479371,8479423,8482061,8475723,8482745,8478439,8471390,8481591,8477043,8471708,8476886,8481061,8482243,8480056,8475727,8473932,8479367,8475324,8479336,8476409,8474563,8476960,8480003,8476910,8468001,8476227,8476967,8480160,8468542,8466378,8474659,8470309,8477369,8471185,8476422,8479613,8475198,8477506,8468535,8481003,8479289,8481827,8480858,8476445,8480031,8479520,8467330,8474683,8475833,8475149,8480979,8467346,8478401,8474168,8474601,8480009,8476877,8480157,8478541,8475878,8476866,8478460,8469490,8474009,8471832,8478495,8473962,8474717,8477314,8480019,8477391,8474012,8470187,8471862,8479981,8475780,8478400,8477971,8480014,8479359,8482705,8479996,8477456,8478928,8466140,8474939,8476278,8478452,8478996,8471767,8474627,8475836,8480063,8479985,8479748,8480172,8480945,8476454,8469467,8476339,8479510,8476457,8471514,8470740,8471768,8480216,8476356,8478069,8480336,8475233,8469623,8474594,8477059,8475243,8475154,8474068,8478451,8480188,8469462,8460542,8477488,8482070,8477343,8477972,8481626,8474094,8471245,8477486,8476827,8477932,8478058,8470871,8482762,8470281,8473482,8474151,8479729,8480977,8476336,8475342,8474134,8474027,8477228,8479413,8478146,8474688,8474573,8475640,8478566,8480946,8471326,8480797,8476326,8473432,8474581,8480761,8475906,8479356,8470222,8481486,8475764,8476947,8471323,8474201,8476191,8476511,8475750,8475188,8477583,8474673,8476522,8479324,8474125,8468635,8470666,8475167,8480878,8476363,8475159,8471352,8471490,8476414,8473492,8470877,8482654,8480845,8476905,8470289,8479368,8478373,8475180,8470613,8482475,8474090,8473431,8481489,8478011,8476988,8476888,8482147,8479411,8473563,8476416,8482712,8478396,8475231,8478967,8476466,8477953,8479987,8478031,8476207,8477962,8480950,8480058,8480221,8478857,8471220,8476931,8475148,8470598,8480144,8477573,8477989,8479980,8471385,8476367,8476545,8472382,8476473,8476934,8480201,8474100,8476971,8475857,8478840,8480873,8481751,8476370,8465166,8474625,8473558,8475730,8474570,8475747,8481683,8479597,8482207,8477082,8465185,8475260,8466285,8477018,8477510,8474066,8480954,8477425,8477471,8469521,8475896,8470156,8475671,8474604,8477507,8477997,8476443,8480158,8475841,8479512,8480222,8480865,8479442,8479398,8477085,8467370,8480252,8476086,8480304,8478367,8475325,8468611,8482740,8475171,8475792,8478432,8471733,8475266,8477497,8475310,8481642,8475829,8471274,8471762,8475155,8467371,8478561,8477964,8480328,8479998,8477473,8478480,8471840,8479557,8479772,8478051,8477320,8480948,8480891,8465058,8481552,8480813,8470153,8481540,8468309,8481530,8474207,8475455,8476792,8476526,8462041,8474870,8484125,8471242,8479986,8478465,8475199,8474498,8476852,8478029,8470159,8471816,8476386,8476400,8470110,8471284,8471817,8471710,8468478,8476890,8479644,8475732,8478178,8480441,8474000,8482399,8480041,8476437,8476536,8474818,8480037,8476391,8476495,8475775,8475758,8479424,8477922,8475196,8459628,8471482,8474743,8478173,8475744,8478528,8475414,8478104,8478436,8480035,8480748,8479376,8479972,8479969,8477289,8480163,8480887,8474135,8478393,8478851,8470065,8477401,8477642,8480798,8473914,8481059,8473658,8479346,8471428,8471504,8482095,8476799,8475834,8476851,8482072,8476919,8475222,8478881,8473446,8470274,8477986,8479373,8476925,8481528,8473712,8471241,8479347,8479026,8470755,8474582,8473573,8478176,8477958,8471436,8475825,8474166,8477380,8471426,8475192,8477455,8470104,8481105,8471240,8469454,8473426,8471956,8475283,8470600,8476404,8481720,8470310,8477913,8482660,8475757,8470816,8477427,8480817,8474571,8476884,8482635,8467910,8470171,8480995,8477428,8476974,8478109,8479339,8476875,8479533,8476857,8474774,8475753,8476822,8471916,8467389,8479578,8474230,8480806,8479511,8474250,8476850,8477445,8477452,8481815,8478397,8479385,8476312,8471719,8462042,8467831,8477009,8476941,8473415,8481754,8476179,8477634,8467396,8474052,8476162,8479425,8479395,8477076,8476401,8471752,8471348,8482250,8480039,8476619,8475718,8475625,8480943,8473514,8476989,8459444,8477335,8480021,8480060,8482452,8473465,8476870,8476779,8475165,8471743,8477249,8474605,8475958,8474837,8480884,8476808,8480940,8480281,8464977,8480883,8474574,8477449,8467493,8478021,8472365,8475332,8476219,8474577,8475181,8470803,8481598,8477666,8471742,8479595,8467496,8473647,8482408,8479415,8481599,8470714,8476855,8480822,8470599,8481161,8470302,8481102,8474520,8474074,8480064,8476477,8477851,8470828,8474138,8473600,8475816,8476177,8480842,8482148,8480005,8467899,8470607,8475274,8476379,8475060,8475235,8479291,8476879,8469547,8474674,8480955,8481629,8470724,8478043,8476897,8482815,8475808,8470121,8477415,8475227,8474061,8474722,8471717,8476922,8473908,8480831,8475650,8476853,8480848,8477544,8479252,8470601,8467338,8477495,8476902,8480823,8474697,8476441,8479362,8476423,8469759,8480292,8466251,8479983,8481650,8478868,8480466,8477472,8474602,8477850,8476856,8473548,8470647,8459670,8474597,8476471,8470685,8481657,8466309,8476874,8474660,8476323,8482624,8482079,8474091,8475321,8480326,8483424,8476396,8480769,8466393,8478136,8475842,8480025,8476805,8473571,8464966,8475759,8476869,8479375,8479970,8475807,8479571,8480855,8470760,8478004,8477974,8479650,8475832,8470920,8478250,8473440,8479372,8469626,8475174,8474130,8471663,8479420,8470719,8479639,8475162,8475436,8483397,8475294,8477715,8480231,8471331,8475197,8476924,8474616,8474618,8474638,8481133,8471657,8479945,8473505,8470617,8462129,8483460,8473646,8475118,8470152,8468526,8468162,8474612,8470151,8479550,8466215,8479999,8476285,8483641,8482634,8477847,8477994,8481043,8466158,8476452,8475766,8475190,8475756,8475735,8480314,8475716,8480836,8474584,8476411,8474744,8474568,8474661,8471856,8472394,8476476,8471718,8480083,8471664,8478874,8480879,8467336,8476435,8481701,8473468,8476206,8475179,8481567,8474707,8478446,8478500,8469542,8465978,8473682,8479755,8474184,8477950,8481517,8478491,8481712,8483515,8474628,8477357,8474551,8471729,8474137,8472424,8467351,8477938,8478415,8481239,8479206,8476147,8470630,8469501,8482062,8479330,8478564,8480184,8481006,8477512,8474579,8479369,8481014,8476020,8475178,8471309,8478055,8479387,8466149,8478468,8471636,8482496,8474598,8465951,8481019,8476431,8481649,8479416,8480164,8468509,8475203,8479268,8474516,8477399,8476300,8479580,8470644,8468085,8471760,8475153,8473584,8457981,8475788,8477205,8471226,8481609,8482176,8474567,8470207,8476304,8478430,8482073,8482641,8465009,8480145,8480871,8475790,8473466,8476465,8468700,8469770,8465025,8474008,8479341,8476418,8480246,8473560,8477943,8477952,8483620,8476464,8477006,8472368,8481433,8477527,8470063,8471269,8477419,8476289,8469555,8479388,8467917,8478859,8473992,8477215,8477504,8475119,8464981,8470662,8467400,8475613,8470854,8466182,8467925,8477509,8477865,8482824,8475236,8479316,8476329,8470775,8480161,8476449,8469665,8479534,8478371,8476303,8475734,8473911,8468513,8474095,8479353,8475150,8479994,8475224,8475246,8479458,8482142,8482222,8478963,8470324,8482699,8471761,8476982,8475762,8480913,8474291,8477359,8477499,8469469,8468778,8475206,8470614,8476953,8467331,8469820,8481426,8480306,8469779,8480147,8474873,8469476,8480011,8471392,8483597,8477460,8474606,8475755,8471693,8471296,8467931,8478476,8464956,8473485,8475264,8478467,8477961,8479421,8474736,8481461,8470054,8481812,8478562,8478506,8470640,8481032,8469639,8480153,8478841,8475868,8475729,8473550,8477589,8467344,8482192,8470839,8458951,8470180,8481585,8475661,8476390,8475182,8475797,8469591,8482089,8469508,8477502,8481575,8476394,8469598,8475279,8475213,8471232,8466142,8481522,8476352,8481541,8478073,8482247,8474610,8482655,8470318,8479348,8470602,8471686,8479338,8460720,8471236,8481600,8475295,8471677,8477453,8481708,8474113,8467452,8471958,8467977,8476470,8477463,8471735,8475917,8477300,8471222,8477462,8477384,8473953,8475648,8469992,8477366,8475980,8475774,8475186,8476279,8479984,8470085,8481572,8473589,8478425,8482861,8480276,8475161,8476450,8477973,8473542,8475223,8467428,8469460,8475795,8477433,8462196,8480072,8482179,8476384,8471388,8470654,8481555,8478844,8467329,8475430,8478477,8480796,8476200,8470699,8480245,8482369,8481147,8477149,8482087,8470317,8478454,8471879,8469619,8475275,8479514,8474698,8473565,8475248,8475815,8481515,8474519,8471748,8474749,8482063,8480439,8479320,8475889,8476478,8468505,8470378,8476388,8468482,8473722,8480847,8477474,8468674,8473700,8481740,8482081,8479333,8470729,8476397,8476158,8479358,8474048,8476403,8481122,8476807,8478842,8474748,8480776,8478224,8477814,8478915,8477916,8476426,8474850,8471811,8465914,8482067,8479402,8477981,8480071,8482133,8479249,8468095,8460492,8479526,8475567,8468114,8459442,8474497,8482125,8471766,8482964,8477073,8469531,8480001,8466333,8473989,8479378,8476043,8467881,8475619,8476407,8467334,8480070,8478062,8481640,8480293,8479991,8478147,8480814]


In [82]:
Player_Bios = get_player_bios(player_id_list)
Player_Bios

Took 1.5641281604766846 seconds to retrieve data for player 8470638
2070 player(s) remain
Took 0.358867883682251 seconds to retrieve data for player 8478402
2069 player(s) remain
Took 1.2299323081970215 seconds to retrieve data for player 8477409
2068 player(s) remain
Took 0.5374813079833984 seconds to retrieve data for player 8479314
2067 player(s) remain
Took 0.5866315364837646 seconds to retrieve data for player 8480835
2066 player(s) remain
Took 0.4208858013153076 seconds to retrieve data for player 8480830
2065 player(s) remain
Took 1.8118538856506348 seconds to retrieve data for player 8475913
2064 player(s) remain
Took 1.9656915664672852 seconds to retrieve data for player 8477933
2063 player(s) remain
Took 0.8850185871124268 seconds to retrieve data for player 8478233
2062 player(s) remain
Took 1.3880207538604736 seconds to retrieve data for player 8475848
2061 player(s) remain
Took 1.2205562591552734 seconds to retrieve data for player 8477210
2060 player(s) remain
Took 0.3520

,Player_ID,Position,Handedness,Birth_Year,Birth_Month,Draft_Year,Draft_Team,Draft_Position
0,8470638,C,Right,1985,July,2003,BOS,45
0,8478402,C,Left,1997,January,2015,EDM,1
0,8477409,C,Left,1995,August,2013,TOR,82
0,8479314,W,Left,1997,December,2016,CGY,6
0,8480835,C,Left,2000,February,2018,CAR,42
...,...,...,...,...,...,...,...,...
0,8481640,C,Left,1998,October,undrafted,undrafted,undrafted
0,8480293,D,Left,1997,April,2017,PHI,196
0,8479991,C,Left,1999,March,2017,VGK,62
0,8478147,W,Left,1996,August,2015,TBL,180


In [84]:
Player_Bios.to_csv('Player_Bios.csv', index=False)